### FILE DETAILS 
requestId= 34952

1. RequestType : AGGREGATED_PROBE_PATH
2. Vehicle Type : ALL
3. BeginDate : 2019-02-02
4. EndDate : 2019-03-04
5. OutputType : TMC
6. RecordCount : 31997952
7. Total TMC Count : 3584

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
df=pd.read_csv('Feb02-March04 TMC\HERE_DA_34952_00000.csv')

In [20]:
df.head(3)

,TMC,DATE-TIME,EPOCH-5MIN,LENGTH,FREEFLOW,SPDLIMIT,COUNT,MEAN,STDDEV,MIN,MAX,CONFIDENCE,PCT-50,PCT-90,GAPFILL
0,D19N00069,2019-02-02 00:00,0,6586,118.1,120.0,55,121.4,8.8,104,130,40,126,130,N
1,D19N00069,2019-02-02 00:05,1,6586,118.1,120.0,26,108.1,7.9,100,125,40,104,125,N
2,D19N00069,2019-02-02 00:10,2,6586,118.1,120.0,12,130.4,1.4,130,135,40,130,134,N


In [5]:
print('Total number of records are : ',df.shape[0])
print('Unique TMC Counts in Dubai are : ',df.TMC.nunique())

Total number of records are :  31997952
Unique TMC Counts in Dubai are :  3584


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31997952 entries, 0 to 31997951
Data columns (total 15 columns):
TMC           object
DATE-TIME     object
EPOCH-5MIN    int64
LENGTH        int64
FREEFLOW      float64
SPDLIMIT      float64
COUNT         int64
MEAN          float64
STDDEV        float64
MIN           int64
MAX           int64
CONFIDENCE    int64
PCT-50        int64
PCT-90        int64
GAPFILL       object
dtypes: float64(4), int64(8), object(3)
memory usage: 3.6+ GB


### Selecting Smaller Datatypes.
Determine and apply the smallest datatype that can fit the range of values

In [21]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [22]:
props, NAlist = reduce_mem_usage(df)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 3661.8750762939453  MB
******************************
Column:  EPOCH-5MIN
dtype before:  int64
dtype after:  uint16
******************************
******************************
Column:  LENGTH
dtype before:  int64
dtype after:  uint16
******************************
******************************
Column:  FREEFLOW
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  SPDLIMIT
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  COUNT
dtype before:  int64
dtype after:  uint16
******************************
******************************
Column:  MEAN
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  STDDEV
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  MIN
dtype before:  int64
dtype after: 

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31997952 entries, 0 to 31997951
Data columns (total 15 columns):
TMC           object
DATE-TIME     object
EPOCH-5MIN    uint16
LENGTH        uint16
FREEFLOW      float32
SPDLIMIT      float32
COUNT         uint16
MEAN          float32
STDDEV        float32
MIN           uint16
MAX           uint16
CONFIDENCE    uint8
PCT-50        uint16
PCT-90        uint16
GAPFILL       object
dtypes: float32(4), object(3), uint16(7), uint8(1)
memory usage: 1.6+ GB


In [23]:
df.head(2)

,TMC,DATE-TIME,EPOCH-5MIN,LENGTH,FREEFLOW,SPDLIMIT,COUNT,MEAN,STDDEV,MIN,MAX,CONFIDENCE,PCT-50,PCT-90,GAPFILL
0,D19N00069,2019-02-02 00:00,0,6586,118.099998,120.0,55,121.400002,8.8,104,130,40,126,130,N
1,D19N00069,2019-02-02 00:05,1,6586,118.099998,120.0,26,108.099998,7.9,100,125,40,104,125,N


In [24]:
df1=df.copy(deep=True)

In [25]:
df1['DateTime']=pd.to_datetime(df1['DATE-TIME'])

In [26]:
df1.columns

Index(['TMC', 'DATE-TIME', 'EPOCH-5MIN', 'LENGTH', 'FREEFLOW', 'SPDLIMIT',
       'COUNT', 'MEAN', 'STDDEV', 'MIN', 'MAX', 'CONFIDENCE', 'PCT-50',
       'PCT-90', 'GAPFILL', 'DateTime'],
      dtype='object')

In [27]:
del df1['DATE-TIME']

In [28]:
df1.columns

Index(['TMC', 'EPOCH-5MIN', 'LENGTH', 'FREEFLOW', 'SPDLIMIT', 'COUNT', 'MEAN',
       'STDDEV', 'MIN', 'MAX', 'CONFIDENCE', 'PCT-50', 'PCT-90', 'GAPFILL',
       'DateTime'],
      dtype='object')

In [31]:
df1.dtypes

TMC                   object
EPOCH-5MIN            uint16
LENGTH                uint16
FREEFLOW             float32
SPDLIMIT             float32
COUNT                 uint16
MEAN                 float32
STDDEV               float32
MIN                   uint16
MAX                   uint16
CONFIDENCE             uint8
PCT-50                uint16
PCT-90                uint16
GAPFILL               object
DateTime      datetime64[ns]
dtype: object

In [38]:
df1.tail()

,TMC,EPOCH-5MIN,LENGTH,FREEFLOW,SPDLIMIT,COUNT,MEAN,STDDEV,MIN,MAX,CONFIDENCE,PCT-50,PCT-90,GAPFILL,DateTime
31997947,D19P09666,283,742,59.0,60.0,0,60.0,0.0,60,60,5,60,60,Y,2019-03-04 23:35:00
31997948,D19P09666,284,742,59.0,60.0,0,60.0,0.0,60,60,5,60,60,Y,2019-03-04 23:40:00
31997949,D19P09666,285,742,59.0,60.0,0,60.0,0.0,60,60,5,60,60,Y,2019-03-04 23:45:00
31997950,D19P09666,286,742,59.0,60.0,2,53.5,0.7,53,54,30,54,54,N,2019-03-04 23:50:00
31997951,D19P09666,287,742,59.0,60.0,0,54.0,0.0,54,54,25,54,54,Y,2019-03-04 23:55:00


### Feather
Feather efficiently stores pandas DataFrame objects on disk. It depends on the Apache Arrow for Python. Feather provides binary columnar serialization for data frames. It is designed to make reading and writing data frames efficient, and to make sharing data across data analysis languages easy.

In [33]:
import feather

In [35]:
df1.to_feather('dubai_TMC.feather')